## Objective
A todos nos gustaría tener una casa y en California sería todo un sueño, pero es este un sueño lograble?...Además, dónde es más caro vivir en el continente o cerca al oceano, dónde hay más casas? Miremos que podemos encontrar en los datos.

In [ ]:
import numpy as np                # linear algebra
import pandas as pd               # data frames
import seaborn as sns             # visualizations
import matplotlib.pyplot as plt   # visualizations
import scipy.stats                # statistics
from sklearn import preprocessing

import os
print(os.listdir("../input"))
#Con esta opción todas los resultados de los comandos se publican en el notebook, ¡muy util!

Miremos que información tiene la base de datos o "dataset", luego de cargarla con la opción "Add Data" y seleccionar el archivo "hausing.csv"

In [ ]:
df = pd.read_csv("../input/housing.csv")#Se define el Dataframe (df) sobre el cual se trabajará

# Print the head of df
print(df.head())

# Print the info of df
print(df.info())

# Print the shape of df
print(df.shape)

Este "dataset" tiene información para 10 variables (*longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, median_house_value y ocean_proximity* ) de 20.640 *census block * del estado de California, recopilada en el Censo de 1990.  [](https://github.com/ageron/handson-ml/tree/master/datasets/housing/)

De estas 10 variables, solo una es no continua y en general la base de datos parece estar balanceada. No obstante, la variable total_bedrooms tiene 3 registros adicionales, ¿por qué? A continuación se verificará que la base efectivamente este balanceada y todas las variables tengan información.

In [ ]:
df.isnull().sum()

La variable con el "problema", efectivamente tiene 207 datos nulos. ¿Qué hacemos con esto?, colocaremos "ceros", en los datos nulos.

In [ ]:
df['total_bedrooms'].fillna(0,inplace = True)
df.isnull().sum()

## Basic Exploratory Data Analysis

In [ ]:
# ¿Qué información tiene la variable categorica?
df.ocean_proximity.value_counts()#cuenta los datos por categoria

#Observemos esta información graficamente
op_count = df['ocean_proximity'].value_counts()
plt.figure(figsize=(10,5))
sns.barplot(op_count.index, op_count.values, alpha=0.7)
plt.title('Ocean Proximity Summary')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Ocean Proximity', fontsize=12)
plt.show()
# housing['ocean_proximity'].value_counts().hist()

In [ ]:
df.iloc[:,~df.columns.isin(['longitude','latitude','ocean_proximity'])].describe()
#Este comando describe las estadisticas básicas para cada una de las variables, exceptuando las señaladas.

Al revisar las estadisticas básicas se tiene que los valores son muy altos, pero debe tener en cuenta que son datos agregados por "districs", que podrían aproximarse a lo que en Colombia llamamos "cuadras". Esto es, no tenemos información por cada vivienda, sino por cuadras de viviendas, por lo cual es preciso circunscribir la información a este contexto.

A continuación, se detallara la información de cada variable para familiarizarnos con la información por bloques o cuadras y entender un poco la alta varaibilidad entre los datos mínimos y máximos para cada variable. Adicionalmente, ¿es acertado comparar la información de las islas con las del continente?

In [ ]:
# Display the histogram to undestand the data
hausing=df.iloc[:,~df.columns.isin(['longitude','latitude'])]
hausing.hist(bins=50, figsize=(20,15))   # pandas DataFrame
plt.show()


In [ ]:
plt.figure(figsize=(15,5))
plt.subplots_adjust(hspace = .25)
plt.subplot(1,2,1)
plt.title('Corelation b/w longtitude and median_house_value')
plt.xlabel('longitude',fontsize=12)
plt.ylabel('median_house_value',fontsize=12)
plt.scatter(df['longitude'].head(100),df['median_house_value'].head(100),color='g')
plt.subplot(1,2,2)
plt.title('Corelation b/w latitude and median_house_value')
plt.xlabel('latitude',fontsize=12)
plt.ylabel('median_house_value',fontsize=12)
plt.scatter(df['latitude'].head(100),df['median_house_value'].head(100),color='r')

A simple vista se pueden observar datos extremos en casi todas las variables, y es posible que haya errores de digitación, ya que en algunos casos estos valores extremos tienen una muy alta frecuencia, teniendo que en los datos censales no son comunes las colas pesadas, o son el mayor dato como en el caso de "housing_median_age" y "median_house_value". 

También se observa que la ubicación espacial, latitud y longitud, parecen tener relación con el valor.

In [ ]:
# Barcharts: are they comparable?
f, axes = plt.subplots(1,2, figsize=(10, 5))
sns.scatterplot(x="median_house_value", y="median_income", data=df, ax=axes[0])
sns.scatterplot(x="median_house_value", y="households", data=df, ax=axes[1])

In [ ]:
f, axes = plt.subplots(1,2, figsize=(10, 5))
sns.scatterplot(x="median_house_value", y="housing_median_age", data=df, ax=axes[0])
sns.scatterplot(x="median_house_value", y="population", data=df, ax=axes[1])

In [ ]:
f, axes = plt.subplots(1,2, figsize=(10, 5))
sns.scatterplot(x="median_house_value", y="total_rooms", data=df, ax=axes[0])
sns.scatterplot(x="median_house_value", y="total_bedrooms", data=df, ax=axes[1])

In [ ]:
f, axes = plt.subplots(1,2, figsize=(10, 5))
sns.scatterplot(x="ocean_proximity", y="median_house_value", data=df, ax=axes[0])

La variable "median_income" parece tener, cómo la intuición lo diria, bastante relación con el "median_house_value". Por su parte la vetustez de las viviendas parece no tener nada que ver con el precio. Es más, pareciera tener una distribución uniforme, al igual que la variable "ocean_proximity".

Asimismo, la población y el número de viviendas (oferta) no parece mostrar una relación extrecha con el valor promedio de las viviendas, asi como el número de espacios y habitaciones en las casas.

**¿Qué informaición nos dan los correlogramas?**

In [ ]:
# Compute the correlation matrix
corr=df.iloc[:,~df.columns.isin(['longitude','latitude', 'ocean_proximity'])].corr()

plt.figure(figsize=(12,7))
sns.heatmap(cbar=False,annot=True,data=corr*100,cmap='Greens')
plt.title('% Corelation Matrix')
plt.show()

Las variables parecen no tener mucha correlación con el valor de las viviendas, pero el ingreso es la que mayor relación tiene. Averiguemos si un ajuste lineal podría ser una opción.

In [ ]:
sns.lmplot(x="median_income",y="median_house_value",data=df)

Pareciera que el ajuste lineal nos dejaría con una enorme porción de datos que no podriamos explicar. ¿Qué información podriamos obtener del valor de las viviendas según su ubicación? A continuación se georeferenciará esta variable.

In [ ]:
# option s: radius of each circle represent the population/100
# option c: color represents the median price
df.plot(kind='scatter', x='longitude', y='latitude', alpha=0.4, 
    #s=df['population']/100, label='population', figsize=(10,7), 
    c='median_house_value', cmap=plt.get_cmap('jet'), colorbar=True)

El mapa de calor del precio parece indicar que las casas de mayor valor se "clusterizan" en una franja cerca al oceano, pero no en toda la costa.

## Data Transformations

Sin lugar a dudas tenemos problemas con los datos extremos. Por lo cual sacaremos de nuestra base los "bloques" con valores muy altos y mucha gente.


In [ ]:
df=df.loc[df['median_house_value']<500001,:]
df=df[df['population']<25000]

hausing=df.iloc[:,~df.columns.isin(['longitude','latitude'])]
hausing.hist(bins=50, figsize=(20,15))   # pandas DataFrame
plt.show()

In [ ]:
No obstante sigue habiendo mucha variación entre los datos. 

# New Features
Para trabajar la enorme varianza de las variables las trabajaremos en escala logaritmica.



In [ ]:
df['median_house_value_log']=np.log1p(df['median_house_value'])


skewed_features=['households','median_income','population', 'total_bedrooms', 'total_rooms']
log_numerical_features=[]
for f in skewed_features:
    df[f + '_log']=np.log1p(df[f])
    log_numerical_features.append(f + '_log')
    
df[log_numerical_features].hist(bins=50, figsize=(10, 10))

Con la transformacion logaritmica mejora mucho el sesgo. Volvamos a hacer la regresión lineal.

In [ ]:
sns.lmplot(x="median_income_log",y="median_house_value_log",data=df)

El ajuste mejoro, pero sigue dejando mucha data por fuera del intervalo de confianza. :(

Sin embargo, es claro que necesitas un ingreso alto para poder vivir en California y que el alto precio depende de la zona para los valores más altos.